In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git # installing latest version of transformers library
!pip install torch # torch
!pip install peft # necessary for finetuning of the large model via LoRA approach
!pip install bitsandbytes # necessary for quantiziation
!pip install evaluate # extension of the transformers library
!pip install datasets # extension of the transformers library
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing 

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding)

import bitsandbytes as bnb

import evaluate
import numpy as np

import random

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

### alternative
from huggingface_hub import login

In [ ]:
dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Group 1: DSSI Summer 2025/Data/summary_banktrak - summary_banktrak.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import random
from datasets import DatasetDict

reduction_rate    = 0.7
# Since there is no 'test' split, we will split the 'train' data
# into a new 'train' and 'test' set.
train_test_split_ratio = 0.3 # 20% for testing

num_total_to_keep = int(reduction_rate * dataset["train"].num_rows)
num_test_to_keep  = int(train_test_split_ratio * num_total_to_keep)
num_train_to_keep = num_total_to_keep - num_test_to_keep


def select_random_indices(dataset, num_to_keep):
    indices = list(range(dataset.num_rows))
    random.shuffle(indices)
    return indices[:num_to_keep]

# Select indices from the original 'train' split for the new 'train' and 'test' splits
all_indices = list(range(dataset["train"].num_rows))
random.shuffle(all_indices)

train_indices = all_indices[num_test_to_keep:num_total_to_keep]
test_indices = all_indices[:num_test_to_keep]


dataset = DatasetDict({
    "train": dataset["train"].select(train_indices),
    "test": dataset["train"].select(test_indices), # Use the original 'train' split to create the new 'test' split
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'company', 'cik', 'item', 'contains_debt_instrument_information'],
        num_rows: 108
    })
    test: Dataset({
        features: ['text', 'company', 'cik', 'item', 'contains_debt_instrument_information'],
        num_rows: 46
    })
})

In [ ]:
model_id  = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(f' Vocab size of the model {model_id}: {len(tokenizer.get_vocab())}')

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

 Vocab size of the model google/gemma-7b-it: 256000


In [ ]:
from datasets import ClassLabel

dataset = dataset.cast_column(
    "contains_debt_instrument_information",
    ClassLabel(names=["True", "False"])  # Replace with your actual class names
)

Casting the dataset:   0%|          | 0/108 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/46 [00:00<?, ? examples/s]

In [ ]:
import torch

def preprocess_function(examples):
# Rename the column and ensure labels are float
  examples["labels"] = examples["contains_debt_instrument_information"].copy()
  examples["labels"] = [float(label) for label in examples["labels"]]
  return tokenizer(examples["text"], truncation=True)

In [ ]:
# This cell will be replaced by the user's code
# tokenized = dataset.map(preprocess_function, batched=True)

tokenized_dataset = {}
for split in ['train','test']:
    # Step 1: Tokenize
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=64),
        batched=True
    )

    # Step 2: Add labels
    tokenized_dataset[split] = tokenized_dataset[split].map(
        lambda x: {"labels": 1 if x["contains_debt_instrument_information"] else 0}
    )

    # Step 3: Clean up
    tokenized_dataset[split] = tokenized_dataset[split].remove_columns(["contains_debt_instrument_information"])

tokenized = tokenized_dataset

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

In [ ]:
id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Convert probabilities to predicted labels
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_use_double_quant=True,  # Use double quantization for potentially higher accuracy (optional)
    bnb_4bit_quant_type="nf4",  # Quantization type (specifics depend on hardware and library)
    bnb_4bit_compute_dtype=torch.bfloat16  # Compute dtype for improved efficiency (optional)
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    quantization_config=bnb_config,
    device_map={"": 0}
)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-7b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
def find_linear_names(model):
    """

You are a financial analyst specializing in SEC filings, specifically tasked with reviewing Form 8-K documents to determine whether they contain debt-related information. Your goal is to classify each document as either "debt-related" (True) or "not debt-related" (False) based on its textual content.

    """
    cls = bnb.nn.Linear4bit

    # Set to store identified layer names
    lora_module_names = set()

    # Iterate through named modules in the model
    for name, module in model.named_modules():
        # Check if the current module is an instance of the 4-bit linear layer class
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

        # Special case: remove 'lm_head' if present
        if 'lm_head' in lora_module_names:
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

# Example usage:
modules = find_linear_names(model)
print(modules)
# ['down_proj', 'gate_proj', 'q_proj', 'o_proj', 'up_proj', 'v_proj', 'k_proj']

['q_proj', 'k_proj', 'v_proj', 'down_proj', 'up_proj', 'o_proj', 'gate_proj']


In [ ]:
lora_config = LoraConfig(
    r=30,  # Reduction factor (lower r means more parameters in the adapter)
    lora_alpha=20,  # Dimensionality of the adapter projection
    target_modules=modules,  # List of modules to apply the LoRA adapter
    lora_dropout=0.1,  # Dropout rate for the adapter
    bias="none",  # Bias configuration for the adapter
    task_type="SEQ_CLS"  # Task type
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

#trainable params: 78,450,688 || all params: 2,584,627,200 || trainable%: 3.0352806006220163

trainable params: 93,763,584 || all params: 8,631,450,624 || trainable%: 1.0863


In [ ]:
training_args = TrainingArguments(
    output_dir="epoch_weights",
    learning_rate=2e-6,  # Learning rate for the optimizer
    per_device_train_batch_size=1,  # Batch size per device
    per_device_eval_batch_size=1,  # Batch size per device for evaluation
    num_train_epochs=7,  # Number of training epochs it was 5 before
    weight_decay=0.1,  # Weight decay for regularization
    eval_strategy='epoch',  # Evaluate after each epoch
    save_strategy="epoch",  # Save model checkpoints after each epoch
    load_best_model_at_end=True,  # Load the best model based on the chosen metric
    push_to_hub=False,  # Disable pushing the model to the Hugging Face Hub
    report_to="none",
    metric_for_best_model='eval_loss',
    seed=42) # Add a seed for reproducibility

In [ ]:
early_stop = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=.0)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],  # Training dataset
    eval_dataset=tokenized["test"],  # Evaluation dataset
    processing_class=tokenizer, # Use processing_class instead of tokenizer
    data_collator=data_collator,  # Data collator for preparing batches
    compute_metrics=compute_metrics,  # Function to calculate evaluation metrics
    callbacks=[early_stop],  # Optional early stopping callback
    # label_names=["labels"] # Explicitly provide label names # This argument is not accepted by Trainer
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,3.078211,0.478261,0.625000,0.540541,0.740741
2,No log,2.809156,0.521739,0.620690,0.580645,0.666667
3,No log,2.694916,0.521739,0.620690,0.580645,0.666667
4,No log,2.603395,0.500000,0.596491,0.566667,0.629630
5,1.692300,2.560314,0.521739,0.607143,0.586207,0.629630
6,1.692300,2.528213,0.521739,0.607143,0.586207,0.629630
7,1.692300,2.518455,0.521739,0.607143,0.586207,0.629630


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=756, training_loss=1.3696992258546214, metrics={'train_runtime': 1241.8465, 'train_samples_per_second': 0.609, 'train_steps_per_second': 0.609, 'total_flos': 2277440286621696.0, 'train_loss': 1.3696992258546214, 'epoch': 7.0})

In [ ]:
def prediction(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to("cuda:0")

    with torch.no_grad():
        outputs = model(**inputs).logits

    # CHANGE: Use softmax instead of sigmoid for single-label classification
    probs = torch.nn.functional.softmax(outputs, dim=-1)[0].cpu().numpy()

    # Get predicted class (index with highest probability)
    predicted_idx = np.argmax(probs)
    predicted_label = id2label[predicted_idx]
    confidence = probs[predicted_idx]

    # Optional: Get sorted labels/probs for debugging
    sorted_indices = np.argsort(probs)[::-1]
    y_sorted_labels = [id2label[i] for i in sorted_indices]
    y_prob_sorted = probs[sorted_indices]

    return predicted_label, confidence  # Or return (y_sorted_labels, y_prob_sorted) if needed

In [ ]:
from transformers import pipeline
import torch.nn.functional as F

# 1. Load dataset
dataset_raw = load_dataset('csv', data_files="/content/drive/MyDrive/Group 1: DSSI Summer 2025/Data/test.csv")
df_test = pd.DataFrame(dataset_raw['train']).rename(columns={'text': 'text'})

# 2. Updated prediction function (single-label binary classification)
def prediction(input_text):
    inputs = tokenizer(input_text,
                     return_tensors="pt",
                     truncation=True,
                     max_length=512).to("cuda:0")

    with torch.no_grad():
        outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=-1)[0].cpu().numpy()

    predicted_idx = np.argmax(probs)
    predicted_label = id2label[predicted_idx]
    confidence = probs[predicted_idx]

    # Return both labels with probabilities for reference
    return [predicted_label, confidence], {id2label[i]: float(probs[i]) for i in range(len(probs))}

# 3. Apply predictions
df_test['pred'] = df_test['text'].apply(lambda x: prediction(x)[0])  # Main prediction
df_test['probs_dict'] = df_test['text'].apply(lambda x: prediction(x)[1])  # Full probabilities

# 4. Split the prediction tuple into columns
df_test[['predicted_label', 'confidence']] = pd.DataFrame(df_test['pred'].tolist(), index=df_test.index)

# 5. Show results
print(f"Test samples: {df_test.shape[0]}")
display(df_test[['text', 'contains_debt_instrument_information', 'predicted_label', 'confidence']].head(10))

# 6. Optional: Calculate accuracy
if 'contains_debt_instrument_information' in df_test.columns:
    accuracy = (df_test['contains_debt_instrument_information'] == df_test['predicted_label']).mean()
    print(f"\nTest Accuracy: {accuracy:.2%}")
    compute_metrics=compute_metrics

Generating train split: 0 examples [00:00, ? examples/s]

Test samples: 1085


,text,contains_debt_instrument_information,predicted_label,confidence
0,\n\n Federated National Holding Company...,True,True,0.999866
1,"\n\n On February 26, 2014, FedFirst Fin...",False,False,0.978594
2,"\n\n On July 12, 2006, All-State Proper...",False,True,0.988280
3,"\n\n-------------\n\nOn February 14, 2006, VoI...",False,True,0.894394
4,"\n\n-------------\n\nOn March 31, 2008, FedFir...",False,False,0.947268
5,\n\n------------------------\n\nThe Registrant...,False,False,0.987322
6,\n\n------------------------------------------...,False,True,1.000000
7,"\n\n(a) On February 2, 2005, the Registrant co...",False,False,0.967800
8,"\n\n(Sonora Resources or the Company), a miner...",False,True,0.999986
9,\n\n1.\n\nMemorandum of Understanding/Loan Agr...,True,True,0.999040



Test Accuracy: 0.00%


In [ ]:
display(df_test[['text', 'contains_debt_instrument_information', 'predicted_label', 'confidence']].head(50))

,text,contains_debt_instrument_information,predicted_label,confidence
0,\n\n Federated National Holding Company...,True,True,0.999866
1,"\n\n On February 26, 2014, FedFirst Fin...",False,False,0.978594
2,"\n\n On July 12, 2006, All-State Proper...",False,True,0.988280
3,"\n\n-------------\n\nOn February 14, 2006, VoI...",False,True,0.894394
4,"\n\n-------------\n\nOn March 31, 2008, FedFir...",False,False,0.947268
5,\n\n------------------------\n\nThe Registrant...,False,False,0.987322
6,\n\n------------------------------------------...,False,True,1.000000
7,"\n\n(a) On February 2, 2005, the Registrant co...",False,False,0.967800
8,"\n\n(Sonora Resources or the Company), a miner...",False,True,0.999986
9,\n\n1.\n\nMemorandum of Understanding/Loan Agr...,True,True,0.999040


In [ ]:

from sklearn.metrics import accuracy_score, f1_score

y_true = df_test['contains_debt_instrument_information'].astype(str).str.upper()
y_pred = df_test['predicted_label'].astype(str).str.upper()

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label='TRUE')  # Specify which class is "positive"

print(f"Accuracy: {accuracy:.2%}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 40.46%
F1 Score: 0.3313


In [ ]:
d¸˛import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 1. Align labels
df_test['true_label'] = df_test['contains_debt_instrument_information'].astype(str).str.upper()
df_test['predicted_label'] = df_test['predicted_label'].astype(str).str.upper()

# 2. Get FP/FN
fp = df_test[(df_test['predicted_label'] == 'TRUE') & (df_test['true_label'] == 'False')]
fn = df_test[(df_test['predicted_label'] == 'False') & (df_test['true_label'] == 'True')]

# 3. Display
print(f"False Positives (FP): {len(fp)}\nSamples:")
print(fp[['text', 'true_label', 'predicted_label', 'confidence']].head(3))

print(f"\nFalse Negatives (FN): {len(fn)}\nSamples:")
print(fn[['text', 'true_label', 'predicted_label', 'confidence']].head(3))

# 4. Confusion Matrix
cm = confusion_matrix(df_test['true_label'], df_test['predicted_label'], labels=['True', 'False'])
disp = ConfusionMatrixDisplay(cm, display_labels=['True', 'False'])
disp.plot()
plt.title('Confusion Matrix')
plt.show()

False Positives (FP): 0
Samples:
Empty DataFrame
Columns: [text, true_label, predicted_label, confidence]
Index: []

False Negatives (FN): 0
Samples:
Empty DataFrame
Columns: [text, true_label, predicted_label, confidence]
Index: []


ValueError: At least one label specified must be in y_true